## 項目11 シーケンスをどのようにスライスするか知っておく

* スライス機能がlist, str, bytesで使える 
    * __getitem__ と __setitem__ を実装すれば使える

In [51]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print('Middle two:   ', a[3:5])
print('All but ends: ', a[1:7])

Middle two:    ['d', 'e']
All but ends:  ['b', 'c', 'd', 'e', 'f', 'g']


In [52]:
# 0を省略できる
assert a[:5] == a[0:5]

In [53]:
# スライス
print(a[:])
print(a[:5])    # 5まで
print(a[:-1])   # 最後の1つ前まで
print(a[4:])
print(a[-3:])   # 最後の3つ前から
print(a[2:5])
print(a[2:-1])
print(a[-3:-1])


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
['a', 'b', 'c', 'd', 'e']
['a', 'b', 'c', 'd', 'e', 'f', 'g']
['e', 'f', 'g', 'h']
['f', 'g', 'h']
['c', 'd', 'e']
['c', 'd', 'e', 'f', 'g']
['f', 'g']


In [54]:
# 要素数を越えていても問題ない
first_twenty_items = a[:20]
last_twenty_items = a[-20:]
assert first_twenty_items == last_twenty_items

In [55]:
# ただし直接アクセスはNG
a[20]

IndexError: list index out of range

In [ ]:
# スライスした結果は参照ではなくコピー
b = a[3:]
print('Before:   ', b)
b[1] = 99
print('After:    ', b)
print('No change:', a)


Before:    ['d', 'e', 'f', 'g', 'h']
After:     ['d', 99, 'f', 'g', 'h']
No change: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']


In [56]:
# スライス側に代入しようとするとその分だけ代入できる！
print('Before ', a)
a[2:7] = [99, 22, 14]
print('After  ', a)

Before  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
After   ['a', 'b', 99, 22, 14, 'h']


In [57]:
# 右辺が長いとリストが追加される
print('Before ', a)
a[2:3] = [47, 11]
print('After  ', a)

Before  ['a', 'b', 99, 22, 14, 'h']
After   ['a', 'b', 47, 11, 22, 14, 'h']


In [58]:
# 複製したいときに[:]を使うテクニック
b = a[:] 
assert b == a and b is not a

In [59]:
# 通常に代入した場合は参照になるので両方変更される
b = a
print('Before ', a)
print('Before ', b)
assert a is b
a[:] = [999, 888, 777]
print('After  ', a)
print('After  ', b)

Before  ['a', 'b', 47, 11, 22, 14, 'h']
Before  ['a', 'b', 47, 11, 22, 14, 'h']
After   [999, 888, 777]
After   [999, 888, 777]


## 項目12 1つの式では、ストライドとスライスを同時に使わない
* スライス：[start:end]で最初と最後を作る
* ストライド：[start:end:stride]でスキップできる

In [60]:
x = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
odds = x[::2]
evens = x[1::2]
print(odds)
print(evens)

['red', 'yellow', 'blue']
['orange', 'green', 'purple']


In [62]:
# [::-1]で逆順にする
x = b'mongoose'
y = x[::-1]
print(y)

b'esoognom'


In [63]:
# Unicodeでも良い
x = '寿司'
y = x[::-1]
print(y)

司寿


In [64]:
# bytesで逆順にしてエンコードすればエラーになる（あたりまえですけども）
w = '寿司'
x = w.encode('utf-8')
y = x[::-1]
z = y.decode('utf-8')
print(z)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb8 in position 0: invalid start byte

In [66]:
# strideを-2にするとどうなるか
x = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print(x[::2])
print(x[::-2])

['a', 'c', 'e', 'g']
['h', 'f', 'd', 'b']


In [69]:
# まぜると紛らわしくなってくる
print(x[2::2])      # 2番目から2飛ばし
print(x[-2::-2])    # -2番目から-2飛ばし
print(x[-2:2:-2])   # -2から2までを-2飛ばし
print(x[2:2:-2])    # 2から2まで-2飛ばし

['c', 'e', 'g']
['g', 'e', 'c', 'a']
['g', 'e']
[]


In [71]:
# わかりやすさ重視で2段階に分ける
# * 先にスライスしたほうがメモリ節約になる
# * メモリを使わずに1飛ばしにしたいなどの要件であればitertoolsのisliceがある
y = x[::2]
z = y[1:-1]
print(y)
print(z)

['a', 'c', 'e', 'g']
['c', 'e']


## 項目13 スライスではなくcatch-allアンパックを使う
アンパックするためには左辺と同じサイズの右辺を用意する必要がある（そうしないと例外が発生する）

In [72]:
car_ages = [0, 9, 4, 8, 7, 20, 19, 1, 6, 15]
car_ages_descending = sorted(car_ages, reverse=True)
oldest, second_oldest = car_ages_descending # 長さが違うのでエラーになる

ValueError: too many values to unpack (expected 2)

In [73]:
# ひとつの解決（スライスする）
# * 悪くはないがインデックスとスライスがあり読みづらい
# * off-by-oneエラーが発生する可能性がある（1～3行目で数値を0,1,2と合わせるコストがある）
oldest = car_ages_descending[0]
second_oldest = car_ages_descending[1]
others = car_ages_descending[2:]
print(oldest, second_oldest, others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]


In [74]:
# よりよい解決（catch-allアンパック）
# * 1つめ、2つめ、その他と文法で示せる
oldest, second_oldest, *others = car_ages_descending #残りをotherに設置できる
print(oldest, second_oldest, others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]


In [75]:
# その他の場所は自由に設置できる
oldest, *others, youngest = car_ages_descending
print(oldest, others, youngest)

*others, second_youngest, youngest = car_ages_descending
print(others, second_youngest, youngest)


20 [19, 15, 9, 8, 7, 6, 4, 1] 0
[20, 19, 15, 9, 8, 7, 6, 4] 1 0


In [76]:
# 少なくとも1つ固定する必要がある
*others = car_ages_descending

SyntaxError: starred assignment target must be in a list or tuple (4026216819.py, line 2)

In [78]:
# 構造が深ければ複数の*を置いてもいいがややこしくなる
# だから、戻り値が多すぎる場合はアンパックしないほうが懸命
car_inventorry = {
    'Downtown': ('Silver Shadow', 'Pinto', 'DMC'),
    'Airport': ('Skyline', 'Viper', 'Gremlin', 'Nova'),
}

((loc1, (best1, *rest1)),
 (loc2, (best2, *rest2))) = car_inventorry.items()
print(f'Best at {loc1} is {best1}, {len(rest1)} others')
print(f'Best at {loc2} is {best2}, {len(rest2)} others')

Best at Downtown is Silver Shadow, 2 others
Best at Airport is Skyline, 3 others
